In [ ]:
#Import modules
import requests
import html5lib
import re
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
from random import randint
from time import sleep

In [ ]:
#Set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)' \
    'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
}

## Step 1: Scrape violation search

In [ ]:
driver = webdriver.Chrome()

In [ ]:
#Search all the search results for every building violation from
#March 20 to November 20, and grabbing their building IDs and violation report IDs

#Set all the different types of violation in the URLs
violation_type = ['AD', 'BL', 'CD', 'CN', 'EE', 'EL', 'HP', 'LL', 'PB', 'PD', 'QL', 'SG', 'SS', 'ZN']
full_list = []
month = 3

#Cycle through the months
while month < 11:
    month_list = []
    loop_count = 0
    while True:
        try:
            violation_type_list = []
            url = f"http://a810-bisweb.nyc.gov/bisweb/ECBBrowseByDateTypeServlet?allviolationtype={violation_type[loop_count]}&allstartdate_month={str(month).zfill(2)}&allstartdate_day=20&allstartdate_year=2020&allenddate_month={str(month + 1).zfill(2)}&allenddate_day=20&allenddate_year=2020&go9=+GO+&requestid=0"
            driver.get(url)
            while True:
                try:
                    #Grab table
                    row_list = []
                    rows = driver.find_elements_by_tag_name("tr")
                    Violation_type = driver.find_elements_by_class_name("maininfo")[1].text
                    for row in rows:
                        row_list.append(row.text)

                    #Make list of dictionaries for each page
                    page_list = []
                    for row in row_list[9:23]:
                        try:
                            BIN = re.findall(r"^(\d+)\s", row)[0]
                        except:
                            BIN = 'Record not found'
                        try:
                            violation_no = re.findall(r"^\d+\s[\d/]+\s([\d\w]+)\s", row)[0]
                        except:
                            violation_no = 'Record not found'
                        row_dict = {
                            'BIN': BIN,
                            'violation_no': violation_no,
                            'Violation_type': Violation_type
                            }
                        page_list.append(row_dict)

                    #Stick the single list on the end of the other lists
                    violation_type_list.extend(page_list)

                    #Go to the next page
                    next_button = driver.find_element_by_xpath('/html/body/center/table[4]/tbody/tr/td[3]/form/input[1]')
                    next_button.click()
                except:
                    break
            month_list.extend(violation_type_list)
            loop_count += 1
        except:
            break
    full_list.extend(month_list)
    month += 1

In [ ]:
#Stick list of dictionaries into a dataframe
df_search = pd.DataFrame(full_list)
df_search.head()

In [ ]:
#Save to CSV
df_search.to_csv('AllBINViolations.csv', index=False)

In [ ]:
#For BIN numbers, get rid of 'Record not found'
bin_array = df_search.BIN.unique()[1:]

In [ ]:
#For violation numbers, get rid of 'Record not found'
violation_no_array = df_search.violation_no.unique()[1:]

## Step 2: Find properties with vacate orders

In [ ]:
#Load from CSV
df_search = pd.read_csv('AllBINViolations.csv')
df_search.head()

In [ ]:
#Find all the URLs for vacate complaints list pages
count = 0
url_list = []
while True:
    try:
        url = f"http://a810-bisweb.nyc.gov/bisweb/PropertyProfileOverviewServlet?bin={bin_array[count]}"
        html = requests.get(url, headers=headers).content
        soup = BeautifulSoup(html, "html.parser")
        try:
            vacant_complaints_url = "http://a810-bisweb.nyc.gov/bisweb/" + soup.find(class_="YellowBanner").next['href']
        except:
            vacant_complaints_url = 'NOT FOUND'
        if vacant_complaints_url != 'NOT FOUND':
            url_list.append(vacant_complaints_url)
        count += 1
        time.sleep(1)
        print(vacant_complaints_url)
    except:
        break
print(url_list)

In [ ]:
#Save to dataframe
df_complaints_url = pd.DataFrame(url_list)
df_complaints_url.head()

In [ ]:
#Save to CSV
df_complaints_url.to_csv('AllComplaintURLs.csv', index=False)

## Step 3: Get URLs for complaint records

In [ ]:
#Load from CSV
df_complaints_url = pd.read_csv('AllComplaintURLs.csv')
df_complaints_url.head()

In [ ]:
#Find URL for the most up-to-date vacate order on properties with full + partial vacate orders

#I kept getting random webpages blocked, so I saved the number of each error so
#that I could later cycle through all the ones that failed on first try
error_count = 0
errors = []
complaint_record_urls = []
for url in df_complaints_url['0']:
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, "html.parser")
    try:
        print("http://a810-bisweb.nyc.gov/bisweb/OverviewForComplaintServlet?" + soup.find('td', class_='content')('a')[0].get('href'))
        complaint_record_urls.append("http://a810-bisweb.nyc.gov/bisweb/OverviewForComplaintServlet?" + soup.find('td', class_='content')('a')[0].get('href'))
    except:
        print(error_count)
        errors.append(error_count)
    error_count += 1
    sleep(randint(1,2))

#Check
print(len(complaint_record_urls))
print(len(errors))

In [ ]:
#Run this until there are no errors left
#[NOTE TO SELF - ADD INTO LOOP ABOVE]
count = 0
for url in df_complaints_url['0'][errors]:
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, "html.parser")
    try:
        print("http://a810-bisweb.nyc.gov/bisweb/" + soup.find('td', class_='content')('a')[0].get('href'))
        complaint_record_urls.append("http://a810-bisweb.nyc.gov/bisweb/" + soup.find('td', class_='content')('a')[0].get('href'))
        del errors[count]
    except:
        print(count, 'Didn\'t work')
        count += 1
    sleep(randint(1,2))

#Check
print(len(complaint_record_urls))
print(len(errors))

In [ ]:
#Save as dataframe
df_complaint_record_url = pd.DataFrame(complaint_record_urls)
df_complaint_record_url.head()

In [ ]:
#Save as CSV
df_complaint_record_url.to_csv('ComplaintRecordURLs.csv', index=False)

## Step 4: Scrape complaint records

In [ ]:
#Load URLs from CSV
df_complaint_record_url = pd.read_csv('ComplaintRecordURLs.csv')
df_complaint_record_url.head()

In [ ]:
#Scrape violation pages for details
error_count = 0
list_of_violations = []
errors = []

url_list = df_complaint_record_url['0']
for url in url_list:
    #Grab URL HTML
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, "html.parser")
    try:
        #Define all scraping variables
        street = soup.find_all('table')[2].find(class_='maininfo').string
        borough = soup.find_all('table')[2].find_all(class_='maininfo')[3].string
        zipcode = soup.find_all('table')[2].find_all(class_='maininfo')[5].string
        BIN = soup.find_all('table')[2].find_all(class_='maininfo')[1]('a')[0].string
        BIN_URL = "http://a810-bisweb.nyc.gov/bisweb/" + soup.find_all('table')[2].find_all(class_='maininfo')[1]('a')[0].get('href')
        table_3_bolds = soup.find_all('table')[3].find_all(class_='content')
        for bold in table_3_bolds:
            if bold.string == 'Category Code:':
                line1 = bold.next.next.next.next
                line2 = line1.next.next.next.next
                line3 = line2.next.next
                category_code = line1.text + " " + line2.text + " " + line3.text
        owner = soup.find_all('table')[6].find_all(class_='content')[-1].string
        date_issued = soup.find_all('table')[7].find_all(class_='content')[3].string
        vacate_type = soup.find_all('table')[7].find_all(class_='content')[3].string
        comments = ''
        table_7_bolds = soup.find_all('table')[7].find_all('b')
        for bold in table_7_bolds:
            if bold.string == 'Comments:':
                comments = bold.next.next.next.next.next
        try:
            complaint = soup.find_all('table')[2].find_all(class_='content')[0].string
        except:
            complaint = ''

        #Set up dictionary
        property_overview = {
            'Street': street,
            'Borough': borough,
            'ZIP': zipcode,
            'BIN': BIN,
            'BIN_URL': BIN_URL,
            'Category_Code': category_code,
            'Owner': owner,
            'Date_Issued': date_issued,
            'Vacate_Type': vacate_type,
            'Comments': comments,
            'Complaint': complaint
        }
        
        list_of_violations.append(property_overview)
        print(property_overview)
    except:
        #Keep track of connection errors to loop through and fix later
        print(error_count)
        errors.append(error_count)
    error_count += 1
    sleep(randint(1,2))

#Check
print(len(list_of_violations))
print(len(errors))

In [ ]:
#Mop up connection errors
#[NOTE TO SELF - INCORPORATE INTO ABOVE LOOP]
count = 0
for url in df_complaint_record_url['0'][errors]:
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, "html.parser")
    try:
        street = soup.find_all('table')[2].find(class_='maininfo').string
        borough = soup.find_all('table')[2].find_all(class_='maininfo')[3].string
        zipcode = soup.find_all('table')[2].find_all(class_='maininfo')[5].string
        BIN = soup.find_all('table')[2].find_all(class_='maininfo')[1]('a')[0].string
        BIN_URL = "http://a810-bisweb.nyc.gov/bisweb/" + soup.find_all('table')[2].find_all(class_='maininfo')[1]('a')[0].get('href')
        table_3_bolds = soup.find_all('table')[3].find_all(class_='content')
        for bold in table_3_bolds:
            if bold.string == 'Category Code:':
                line1 = bold.next.next.next.next
                line2 = line1.next.next.next.next
                line3 = line2.next.next
                category_code = line1.text + " " + line2.text + " " + line3.text
        owner = soup.find_all('table')[6].find_all(class_='content')[-1].string
        date_issued = soup.find_all('table')[7].find_all(class_='content')[3].string
        vacate_type = soup.find_all('table')[7].find_all(class_='content')[3].string
        comments = ''
        table_7_bolds = soup.find_all('table')[7].find_all('b')
        for bold in table_7_bolds:
            if bold.string == 'Comments:':
                comments = bold.next.next.next.next.next
        try:
            complaint = soup.find_all('table')[2].find_all(class_='content')[0].string
        except:
            complaint = ''

        #Set up dictionary
        property_overview = {
            'Street': street,
            'Borough': borough,
            'ZIP': zipcode,
            'BIN': BIN,
            'BIN_URL': BIN_URL,
            'Category_Code': category_code,
            'Owner': owner,
            'Date_Issued': date_issued,
            'Vacate_Type': vacate_type,
            'Comments': comments,
            'Complaint': complaint
        }
        
        list_of_violations.append(property_overview)
        print(property_overview)
        #Remove successful list entry
        del errors[count]
    except:
        #Note the remaining errors
        print(count, 'Didn\'t work')
        count += 1
    sleep(randint(1,2))

#Check
print(len(list_of_violations))
print(len(errors))

In [ ]:
#Save as dataframe
df_list_of_violations = pd.DataFrame(list_of_violations)
df_list_of_violations.head()

In [ ]:
#Save as CSV
df_list_of_violations.to_csv('VacateOrderComplaintRecords.csv', index=False)

## Step 5: Cleaning the dataframe

In [ ]:
#Load records from CSV
df_violations = pd.read_csv('VacateOrderComplaintRecords.csv')
df_violations.head()

In [ ]:
#Clean streets
df_violations.Street = df_violations.Street.replace(r'Complaint at:', r'', regex=True).str.title().str.strip()

In [ ]:
#Clean boroughs
df_violations.Borough = df_violations.Borough.replace(r'Borough:', r'', regex=True).str.title().str.strip()

In [ ]:
#Clean ZIP
df_violations.ZIP = df_violations.ZIP.replace(r'ZIP:', r'', regex=True).str.strip()

In [ ]:
#Clean BIN
df_violations.BIN = df_violations.BIN.astype(str, errors='ignore').str[:-2]

In [ ]:
#Clean category code
df_violations.Category_Code = df_violations.Category_Code\
.str.replace(r'^[\d\w]{2}', '')\
.str.replace(r'[\n]', '|')\
.str.replace('\|\|', '| ')\
.str.strip()\
.str.replace(r'\s:', ':')\
.str.strip()\
.str.replace(r'\|$', '')\
.str.strip()\
.str.title()

In [ ]:
#Clean up Owner - there are *lots* of inconsistencies in the data here
df_violations.Owner = df_violations.Owner.str.strip().replace({'X X X': 'NO RECORD', 'X X': 'NO RECORD', 'X': 'NO RECORD', 'NAME NOT ON FILE': 'NO RECORD', 'UNAVAILABLE OWNER': 'NO RECORD', '* *': 'NO RECORD'})

pd.set_option("max_rows", 1000)
df_violations.Owner = df_violations.Owner.str.title()\
.str.replace('Llc', 'LLC')\
.str.replace('Lp', 'LP')\
.str.replace('Nyc', 'NYC')\
.str.replace(r'\d(Th)', 'th')\
.str.replace(r'\d(Nd)', 'nd')\
.str.replace('Ll', 'LL')\
.str.replace(r'\s\s+', ' ')\
.str.replace(r'(\s)[,\.]', '')\
.str.replace(r'Iii', 'III')\
.str.replace(r',$', '')

In [ ]:
#Clean date
df_violations.Date_Issued = df_violations.Date_Issued.str.extract(r'^(\d\d/\d\d/\d\d\d\d)')

In [ ]:
#Clean vacate type
df_violations.Vacate_Type = df_violations.Vacate_Type.str.extract(r'^\d\d/\d\d/\d\d\d\d - Y\d - ([\w]+ [\w]+)')

df_violations.Vacate_Type = df_violations.Vacate_Type.str.title()

In [ ]:
#Clean comments
df_violations.Comments = df_violations.Comments\
.str.replace('Nypd', 'NYPD')\
.str.replace('Nyfd', 'NYFD')\
.str.replace(r'\s\s+', ' ')\
.str.replace(r'(,)$', '.')\
.str.replace(r'()$', '.')\
.str.replace('\.+', '.')\
.str.strip()

In [ ]:
#Clean complaints
df_violations.Complaint = df_violations.Complaint\
.str.replace(r'^(Re:\s+)', '')\
.str.replace(r'\s\s+', ' ')\
.str.replace(r'(,)$', '.')\
.str.replace(r'()$', '.')\
.str.replace('\.+', '.')\
.str.strip()

In [ ]:
#Convert date to datetime
df_violations.Date_Issued = pd.to_datetime(df_violations.Date_Issued, format='%m/%d/%Y').to_frame()
df_violations.Date_Issued.describe

In [ ]:
#Save as CSV
df_violations.to_csv('VacateOrderComplaintRecords_Cleaned1.csv', index=False)

## Step 6: filter out pandemic vacate orders

In [ ]:
#Load cleaned violations from CSV
df_violations = pd.read_csv('VacateOrderComplaintRecords_Cleaned1.csv')
df_violations.head()

In [ ]:
#Convert ZIP to string, remove .0
df_violations['ZIP'] = df_violations['ZIP'].astype(str).str[:-2].astype(str)
df_violations['ZIP'].describe

In [ ]:
df_quarantine_violations = df_violations[df_violations['Date_Issued'] > '2020-03-20'].reset_index()
df_quarantine_violations.head()

In [ ]:
#Save cleaned database as CSV
df_quarantine_violations.to_csv('VacateOrderComplaintRecords_Quarantine.csv', index=False)

## Step 7: Mopping up

In [ ]:
#Messed up and added an errant index at some point - taking it out here
df_qv = pd.read_csv('VacateOrderComplaintRecords_Quarantine.csv')
df_qv = df_qv.drop('index', axis=1)
df_qv.to_csv('VacateOrderComplaintRecords_Quarantine.csv', index=False)

In [ ]:
#Fixing vacate order edge cases
df_qv.Vacate_Type = df_qv.Vacate_Type.str.replace('Vacate Order', 'Vacate Order Partially Rescinded')

In [ ]:
df_qv.to_csv('VacateOrderComplaintRecords_Quarantine.csv', index=False)

## Bit of analysis

In [1]:
#Import modules
import requests
import html5lib
import re
import time
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
from random import randint
from time import sleep

In [2]:
#Set headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)' \
    'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
}

In [4]:
#Load cleaned violations from CSV
df_qv = pd.read_csv('VacateOrderComplaintRecords_Quarantine.csv')
df_qv.head(3)

Street   Borough    ZIP      BIN  \
0  51 East Mount Eden Avenue     Bronx  10452  2008149   
1        1072 East 39 Street  Brooklyn  11210  3213883   
2           138-11 97 Avenue    Queens  11435  4214086   

                                             BIN_URL  \
0  http://a810-bisweb.nyc.gov/bisweb/PropertyProf...   
1  http://a810-bisweb.nyc.gov/bisweb/PropertyProf...   
2  http://a810-bisweb.nyc.gov/bisweb/PropertyProf...   

                                       Category_Code                   Owner  \
0  Building Shaking/Vibrating/Struct Stability Af...  49 Mt. Eden Realty LLC   
1  Illegal Conversion | Illegal Conversion: Resid...          Gunter, Joseph   
2  Illegal Conversion | Illegal Conversion: Resid...       Niyazova, Susanna   

  Date_Issued     Vacate_Type  \
0  23/03/2020     Full Vacate   
1  26/03/2020  Partial Vacate   
2  16/04/2020  Partial Vacate   

                                            Comments  \
0              ECB 35433195K REPLACED WIE 35433188M.   
1  OBSERVED OCCUPIED CONTRARY TO THE C OF O # 916...   
2  2 SUMMONSES ISSUED AT CELLAR LEVEL FOR WORK W/...   

                                           Complaint  
0  FDNY REQUESTS A STRUCTURAL STABILITY INSPECTIO...  
1  APARTMENT IS IN THE BASEMENT AND HAS BEEN ILLE...  
2                 BASEMENT TURNED INTO AN APARTMENT.

In [5]:
#Any repeat offenders?
df_qv.groupby('Owner').Owner.count().sort_values(ascending=False).head(5)

Owner
No Record                    13
Deen, Nouras M                2
Zheng, Yi Shui                1
Fjh Rlty Inc                  1
Chondrogiannis, Alexandra     1
Name: Owner, dtype: int64

In [6]:
#Most common types of violation
df_qv.groupby('Category_Code').Category_Code.count().sort_values(ascending=False).head(10)
#Lots similar to each other - figure out how to aggregate
#Maybe split by pipe and then have list entries? Not sure how this
#would work in terms of presenting in a dataframe.

Category_Code
Illegal Conversion | Illegal Conversion: Residential Space                                                     32
Building Shaking/Vibrating/Struct Stability Affected                                                           31
Illegal Conversion                                                                                              9
Failure To Maintain                                                                                             8
Building Shaking/Vibrating/Struct Stability Affected | Unstable Building: Leaning - Construction/Demolition     7
Egress - Locked/Blocked/Improper/No Secondary Means                                                             7
Debris/Building - Falling Or In Danger Of Falling                                                               4
Building Shaking/Vibrating/Struct Stability Affected | Unstable Building: Shaking/Vibrating - Construction      3
Sro - Illegal Work/No Permit/Change In Occup-Use | Single Room Occupancy: 

In [7]:
#Violations by borough
df_qv.groupby('Borough').Borough.count().sort_values(ascending=False)

Borough
Brooklyn         49
Queens           36
Bronx            17
Manhattan        12
Staten Island     8
Name: Borough, dtype: int64

In [8]:
#Violations by ZIP
df_qv.groupby('ZIP').ZIP.count().sort_values(ascending=False).head(10)

ZIP
10459    5
11207    5
11213    4
11358    4
11212    3
10302    3
10301    3
11368    3
11216    3
11222    3
Name: ZIP, dtype: int64

In [9]:
#Types of vacate order
df_qv.groupby('Vacate_Type').Vacate_Type.count().sort_values(ascending=False)

Vacate_Type
Partial Vacate                      94
Full Vacate                         26
Vacate Order Partially Rescinded     2
Name: Vacate_Type, dtype: int64